In [1]:
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from unidecode import unidecode
import unidecode


In [2]:
import geopandas as gpd

In [3]:
import numpy as np
from shapely.geometry import Point, Polygon

In [ ]:
pip install --upgrade --force-reinstall rtree


In [ ]:
!pip install geopandas


In [ ]:
!pip install geopandas

In [ ]:
# Cargar el archivo CSV
datos_csv = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones.csv')


In [ ]:
# Cambiar los nombres de las columnas 'Unnamed: 0' y 'Unnamed: 1'
datos_csv = datos_csv.rename(columns={'Unnamed: 0': 'ciudad', 'Unnamed: 1': 'detenciones'})

datos_csv = datos_csv.dropna()

# Verificar los cambios mostrando las primeras filas del DataFrame
print(datos_csv.head())


In [ ]:
# Inicializamos la columna 'estado' con NaN
datos_csv['estado'] = pd.NA

# Almacenamos temporalmente la etiqueta de estado actual
etiqueta_estado_actual = pd.NA

# Paso 2: Asignar la etiqueta de estado a la nueva columna 'estado'
for idx, tiene_etiqueta in es_etiqueta_estado.items():
    if tiene_etiqueta:  # Si la fila actual es una etiqueta de estado
        etiqueta_estado_actual = datos_csv.at[idx, 'ciudad']  # Actualiza la etiqueta de estado
    datos_csv.at[idx, 'estado'] = etiqueta_estado_actual  # Asigna la etiqueta de estado actual a la columna 'estado'

# Opcionalmente, puedes querer limpiar los valores de 'estado' para quitar la parte ', Ags.' o similar
datos_csv['estado'] = datos_csv['estado'].str.replace(',.*', '', regex=True)

# Verificación: Muestra las primeras filas para ver el resultado
print(datos_csv.head(10))


In [ ]:
# Crear un filtro para identificar filas donde 'ciudad' contiene ',' o '.'
filtro = datos_csv['ciudad'].str.contains(',', regex=False) | datos_csv['ciudad'].str.contains('\.', regex=True)

# Aplicar el filtro para mantener solo las filas que cumplen con la condición
datos_csv_filtrado = datos_csv[filtro]



In [ ]:
# Cambiar las etiquetas de 'Mexico' a 'Estado de Mexico' en la columna 'estado'
datos_csv['estado'] = datos_csv['estado'].replace('Mexico', 'Estado de Mexico')

In [ ]:
# Primero, separa 'ciudad' por la coma y toma solo la parte antes de la coma
datos_csv['ciudad'] = datos_csv['ciudad'].str.split(',').str[0]

# Luego, concatena esa parte con el valor correspondiente de 'estado'
datos_csv['ciudad'] = datos_csv['ciudad'] + ', ' + datos_csv['estado']

# Verificación: Muestra las primeras filas para ver el resultado
print(datos_csv.head())

In [ ]:
datos_csv.shape

In [ ]:
import pandas as pd
from unidecode import unidecode

# Corrección en la importación de unidecode
def quitar_acentos(valor):
    if isinstance(valor, str):  # Solo aplica a cadenas de texto
        return unidecode(valor)  # Uso correcto de unidecode
    return valor

# Aplicar la función quitar_acentos a todas las columnas del DataFrame
for columna in datos_csv.columns:
    datos_csv[columna] = datos_csv[columna].apply(quitar_acentos)

# Verificación: Mostrar las primeras filas para asegurarse de que los acentos han sido quitados
print(datos_csv.head())


In [ ]:
# Definir las listas de estados para cada ruta, asegurándose de que todas las entradas están correctamente formateadas como strings
estados_ruta_golfo = ['Tabasco', 'Nuevo Leon', 'Veracruz', 'Tamaulipas']
estados_ruta_pacifico = ['Guerrero', 'Michoacan', 'Jalisco', 'Nayarit', 'Sinaloa', 'Sonora', 'Baja California', 'Baja California Sur','Colima']
estados_ruta_centro = ['San Luis Potosi','Estado de Mexico', 'Tlaxcala', 'Ciudad de Mexico', 'Puebla', 'Morelos', 'Hidalgo', 'Zacatecas', 'Durango', 'Chihuahua', 'Coahuila', 'Queretaro', 'Aguascalientes', 'Guanajuato']
ruta_sur = ['Chiapas', 'Oaxaca', 'Tabasco', 'Campeche', 'Quintana Roo', 'Yucatan']

# Actualizar la función para asignar la ruta basada en el estado
def asignar_ruta(estado):
    estado = estado.strip()  # Eliminar espacios en blanco al principio y al final
    if estado in estados_ruta_golfo:
        return 'Ruta del Golfo'
    elif estado in estados_ruta_pacifico:
        return 'Ruta del Pacifico'
    elif estado in estados_ruta_centro:
        return 'Ruta Centro'
    elif estado in ruta_sur:
        return 'Ruta Sur'
    else:
        return 'Ruta no identificada'  # Para estados que no coinciden con las listas

# Aplicar la función al DataFrame para crear una nueva columna 'Ruta'
datos_csv['ruta'] = datos_csv['estado'].apply(asignar_ruta)

In [ ]:
# Reordenar las columnas para colocar 'Estado' después de 'Ciudad'
column_order = ['ciudad', 'estado', 'ruta','detenciones']

# Aplicar el nuevo orden de columnas al DataFrame
datos_csv = datos_csv[column_order]

In [ ]:
# Obtenemos los valores únicos de la columna 'ruta'
valores_unicos_ruta = datos_csv['ruta'].unique()

# Imprimimos los valores únicos
print(valores_unicos_ruta)

In [ ]:
# Initialize the geolocator using OSM Nominatim
geolocator = Photon(user_agent="measurement", timeout=10)

# Define a function to get latitude and longitude
def get_lat_lon(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return '', ''  # Return empty strings if location is not found
    except AttributeError:
        return '', ''

# Apply the function to the 'City' column and split the results into 'Latitude' and 'Longitude' columns
datos_csv[['latitud', 'longitud']] = datos_csv['ciudad'].apply(lambda x: pd.Series(get_lat_lon(x)))

In [ ]:
# Verificar los cambios mostrando las primeras filas del DataFrame filtrado
print(datos_csv.head(20))

In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
datos_csv.to_csv(ruta_archivo_csv, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv}")


In [ ]:
# Cargar el archivo CSV
datos_csv1 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv')


In [ ]:
datos_csv1

In [ ]:
# Convertir la columna 'detenciones' a entero
datos_csv1['detenciones'] = pd.to_numeric(datos_csv1['detenciones'], errors='coerce').fillna(0).astype(int)

In [ ]:
# Identificar los índices de la primera fila de cada categoría en 'estado'
indices_a_eliminar = datos_csv1.groupby('estado').head(1).index

# Eliminar esas filas del DataFrame
datos_csv1 = datos_csv1.drop(indices_a_eliminar)




In [ ]:
# Cambiar latitud y longitud para "Zacatecas, Zacatecas"
datos_csv1.loc[datos_csv1['ciudad'] == 'Zacatecas, Zacatecas', 'latitud'] = 22.771727298316314
datos_csv1.loc[datos_csv1['ciudad'] == 'Zacatecas, Zacatecas', 'longitud'] = -102.57893967251229

# Cambiar latitud y longitud para "Durango, Durango"
datos_csv1.loc[datos_csv1['ciudad'] == 'Durango, Durango', 'latitud'] = 24.03507526470482
datos_csv1.loc[datos_csv1['ciudad'] == 'Durango, Durango', 'longitud'] = -104.66366132513203

# Verificar los cambios
print(datos_csv1[datos_csv1['ciudad'].isin(['Zacatecas, Zacatecas', 'Durango, Durango'])])


In [ ]:
# Verificar los cambios
print(datos_csv1.head())


In [ ]:
# Agregar la columna 'tipo' con la etiqueta 'Presencia Migración Irregular' para todas las filas
datos_csv1['tipo'] = 'Presencia Migración Irregular'

# Verificar los cambios mostrando las primeras filas del DataFrame
print(datos_csv1.head())


In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv1 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
datos_csv1.to_csv(ruta_archivo_csv1, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv1}")


### Grid Shape

Para crear un grid con polígonos geoespaciales y asignar a cada punto según su latitud y longitud a un polígono correspondiente, se puede utilizar la biblioteca geopandas en Python, que extiende las capacidades de pandas para trabajar con datos espaciales. También usaremos shapely para manipular y analizar datos geométricos planos, y numpy para manejar operaciones numéricas.



Crear el Grid de Polígonos: Primero, generamos un conjunto de polígonos que representen cada celda del grid sobre el área de interés (en este caso, México). Cada polígono puede ser identificado de manera única, por ejemplo, usando una convención de nomenclatura como PG1, PG2, etc.

In [17]:
def crear_grid(xmin, xmax, ymin, ymax, ancho_celda, alto_celda):
    cols = list(np.arange(xmin, xmax, ancho_celda))
    rows = list(np.arange(ymin, ymax, alto_celda))
    poligonos = []
    ids = []

    id_contador = 1  # Iniciar contador para los ID de polígonos
    for x in cols:
        for y in rows:
            poligonos.append(Polygon([(x, y), (x+ancho_celda, y), (x+ancho_celda, y-alto_celda), (x, y-alto_celda)]))
            ids.append(f"PG{id_contador}")  # Asignar un ID único a cada polígono
            id_contador += 1

    grid = gpd.GeoDataFrame({'ID_Poligono': ids, 'geometry': poligonos})
    return grid

# Generar el grid con los límites aproximados de México y celdas de 1 grado por 1 grado
xmin, ymin, xmax, ymax = -118, 14, -86, 33
grid = crear_grid(xmin, xmax, ymin, ymax, 1, 1)

# Ahora, 'grid' es un GeoDataFrame que contiene el 'ID_Poligono' y su 'geometry'
print(grid.head())

  ID_Poligono                                           geometry
0         PG1  POLYGON ((-118.00000 14.00000, -117.00000 14.0...
1         PG2  POLYGON ((-118.00000 15.00000, -117.00000 15.0...
2         PG3  POLYGON ((-118.00000 16.00000, -117.00000 16.0...
3         PG4  POLYGON ((-118.00000 17.00000, -117.00000 17.0...
4         PG5  POLYGON ((-118.00000 18.00000, -117.00000 18.0...


In [7]:
# Cargar el archivo CSV
datos_csv2 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv')


In [4]:
# Paso 1: Crear el Grid de Polígonos
def crear_grid(xmin, xmax, ymin, ymax, ancho_celda, alto_celda):
    cols = list(np.arange(xmin, xmax, ancho_celda))
    rows = list(np.arange(ymin, ymax, alto_celda))
    poligonos = []

    for x in cols:
        for y in rows:
            poligonos.append(Polygon([(x, y), (x+ancho_celda, y), (x+ancho_celda, y-alto_celda), (x, y-alto_celda)]))
    
    grid = gpd.GeoDataFrame({'geometry': poligonos})
    grid['ID'] = ['PG' + str(i) for i in range(1, len(grid)+1)]
    return grid

# Asumiendo que tienes los límites de México, aquí un ejemplo con valores hipotéticos
xmin, ymin, xmax, ymax = -118, 14, -86, 33  # Límites aproximados de México
grid = crear_grid(xmin, xmax, ymin, ymax, 1, 1)  # Celdas de 1 grado por 1 grado


In [8]:
# Paso 2: Asignar Puntos a Polígonos
puntos = [Point(xy) for xy in zip(datos_csv2.longitud, datos_csv1.latitud)]
geo_df = gpd.GeoDataFrame(datos_csv2, geometry=puntos)


In [9]:
# Realizar la asignación
geo_df = gpd.sjoin(geo_df, grid, how='left', op='within')

/Users/pablouriarte/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [12]:
# Paso 3: Actualizar DataFrame
datos_csv2['ID_Poligono'] = geo_df['ID']

# Muestra el resultado
print(datos_csv2.head())

                                      ciudad           estado  \
0             Aguascalientes, Aguascalientes   Aguascalientes   
1                      Cosio, Aguascalientes   Aguascalientes   
2                Jesus Maria, Aguascalientes   Aguascalientes   
3  San Francisco de los Romo, Aguascalientes   Aguascalientes   
4                  Ensenada, Baja California  Baja California   

                ruta  detenciones    latitud    longitud  \
0        Ruta Centro          423  22.000000 -102.500000   
1        Ruta Centro           98  22.370515 -102.312009   
2        Ruta Centro           20  21.934555 -102.468452   
3        Ruta Centro           18  22.074628 -102.270713   
4  Ruta del Pacifico           29  31.865889 -116.602983   

                            tipo ID_Poligono  
0  Presencia Migración Irregular       PG295  
1  Presencia Migración Irregular       PG295  
2  Presencia Migración Irregular       PG294  
3  Presencia Migración Irregular       PG295  
4  Presencia 

In [19]:
# Asegurándose de que datos_csv2 es un GeoDataFrame
if not isinstance(datos_csv2, gpd.GeoDataFrame):
    # Si datos_csv2 ya tiene una columna 'geometry', úsala; de lo contrario, se necesita definir una
    if 'geometry' in datos_csv2.columns:
        datos_csv2 = gpd.GeoDataFrame(datos_csv2, geometry='geometry')
    else:
        # Aquí se asume que no hay una columna 'geometry' existente y que quizás no necesites definir una en este momento
        datos_csv2 = gpd.GeoDataFrame(datos_csv2)

# Realizar el merge (unión) entre datos_csv2 y grid usando 'ID_Poligono' como clave
datos_csv2 = datos_csv2.merge(grid[['ID_Poligono', 'geometry']], on='ID_Poligono', how='left')


In [20]:
datos_csv2.head()

,ciudad,estado,ruta,detenciones,latitud,longitud,tipo,ID_Poligono,ID,geometry_x,geometry_y,geometry
0,"Aguascalientes, Aguascalientes",Aguascalientes,Ruta Centro,423,22.000000,-102.500000,Presencia Migración Irregular,PG295,PG295,"POLYGON ((-103.00000 23.00000, -102.00000 23.0...","POLYGON ((-103.00000 23.00000, -102.00000 23.0...","POLYGON ((-103.00000 23.00000, -102.00000 23.0..."
1,"Cosio, Aguascalientes",Aguascalientes,Ruta Centro,98,22.370515,-102.312009,Presencia Migración Irregular,PG295,PG295,"POLYGON ((-103.00000 23.00000, -102.00000 23.0...","POLYGON ((-103.00000 23.00000, -102.00000 23.0...","POLYGON ((-103.00000 23.00000, -102.00000 23.0..."
2,"Jesus Maria, Aguascalientes",Aguascalientes,Ruta Centro,20,21.934555,-102.468452,Presencia Migración Irregular,PG294,PG294,"POLYGON ((-103.00000 22.00000, -102.00000 22.0...","POLYGON ((-103.00000 22.00000, -102.00000 22.0...","POLYGON ((-103.00000 22.00000, -102.00000 22.0..."
3,"San Francisco de los Romo, Aguascalientes",Aguascalientes,Ruta Centro,18,22.074628,-102.270713,Presencia Migración Irregular,PG295,PG295,"POLYGON ((-103.00000 23.00000, -102.00000 23.0...","POLYGON ((-103.00000 23.00000, -102.00000 23.0...","POLYGON ((-103.00000 23.00000, -102.00000 23.0..."
4,"Ensenada, Baja California",Baja California,Ruta del Pacifico,29,31.865889,-116.602983,Presencia Migración Irregular,PG38,PG38,"POLYGON ((-117.00000 32.00000, -116.00000 32.0...","POLYGON ((-117.00000 32.00000, -116.00000 32.0...","POLYGON ((-117.00000 32.00000, -116.00000 32.0..."


Asignar Puntos a Polígonos: Luego, para cada punto dado por sus coordenadas de latitud y longitud en datos_csv1, determinamos en qué polígono del grid se encuentra y asignamos el identificador de ese polígono al punto.


Actualizar DataFrame: Finalmente, agregamos una nueva columna al DataFrame datos_csv1 para almacenar el identificador del polígono del grid al que pertenece cada punto.

In [21]:
# Obtener el conteo de los valores únicos en la columna 'ID_Poligono'
conteo_id_poligono = datos_csv2['ID_Poligono'].value_counts()

# Mostrar el conteo
print(conteo_id_poligono)



ID_Poligono
PG368    36
PG349    19
PG336    15
PG404    15
PG405    12
         ..
PG280     1
PG113     1
PG338     1
PG114     1
PG276     1
Name: count, Length: 117, dtype: int64
